<a href="https://colab.research.google.com/github/DLOHai/Burgundymusic/blob/master/burgundy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Sep  5 12:30:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!mkdir custom dataset logs deprecated midi_processor result

In [0]:
!mkdir ./dataset/midi ./dataset/processed ./dataset/scripts

In [0]:
!mkdir ./midi_processor/bin

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.0MB 9.4MB/s 
     |████████████████████████████████| 102kB 37.8MB/s 
     |████████████████████████████████| 102kB 42.0MB/s 
     |████████████████████████████████| 133kB 71.9MB/s 
     |████████████████████████████████| 5.6MB 54.7MB/s 
     |████████████████████████████████| 153kB 48.7MB/s 
     |████████████████████████████████| 2.2MB 46.9MB/s 
     |████████████████████████████████| 92kB 38.5MB/s 
     |████████████████████████████████| 143kB 54.4MB/s 
     |████████████████████████████████| 552kB 39.3MB/s 
     |████████████████████████████████| 266kB 57.2MB/s 
     |████████████████████████████████| 92kB 32.3MB/s 
     |████████████████████████████████| 829kB 56.4MB/s 
     |████████████████████████████████| 71kB 32.0MB/s 
     |████████████████████████████████| 450kB 54.3MB/s 
     |████████████████████████████████| 61kB 29.4MB/s 
     |████████████████████████████████| 2.2MB 54.4MB/s 
     |████████████████████████████████| 2.8MB 66.2MB/

In [0]:
!python train.py --save_path='./result/'

<class Data has "201" files>
2019-09-03 12:36:26.330223: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-03 12:36:26.393007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-03 12:36:26.393566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2019-09-03 12:36:26.401418: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-09-03 12:36:26.565343: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-09-03 12:36:26.651811: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dyna

In [0]:
from model import MusicTransformerDecoder
from custom.layers import *
from custom import callback
import params as par
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
import datetime
import argparse
from midi_processor.processor import decode_midi, encode_midi

In [0]:
# set arguments
max_seq = 2048
load_path = './result'
mode = 'dec'

In [0]:
beam = 4
length = 2048

In [18]:
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
gen_log_dir = 'logs/mt_decoder/generate_'+current_time+'/generate'
gen_summary_writer = tf.summary.create_file_writer(gen_log_dir)


print(">> generate with decoder wise... beam size is {}".format(beam))
mt = MusicTransformerDecoder(loader_path=load_path)

inputs = encode_midi('./dataset/midi/chopin_canon_in_F_minor.mid')
print("♥♥♥♥♥ inputs are generated, length: ",len(inputs))
length = len(inputs)

>> generate with decoder wise... beam size is 4


W0905 13:16:05.759974 139657985136512 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0905 13:16:05.761014 139657985136512 util.py:244] Unresolved object in checkpoint: (root).grad
W0905 13:16:05.761738 139657985136512 util.py:244] Unresolved object in checkpoint: (root).fc.kernel
W0905 13:16:05.762444 139657985136512 util.py:244] Unresolved object in checkpoint: (root).fc.bias
W0905 13:16:05.763216 139657985136512 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0905 13:16:05.764608 139657985136512 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0905 13:16:05.765325 139657985136512 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0905 13:16:05.766575 139657985136512 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0905 13:16:05.767721 139657985136512 util.py:244] Unresolved object in checkpoint: (root).Decoder.embedding.embeddings
W0905 13:16:05.768798 139657985136512 util.py:244] Unr

[Note(start=0.000000, end=0.250000, pitch=61, velocity=84), Note(start=0.218750, end=0.567708, pitch=60, velocity=84), Note(start=0.619792, end=0.828125, pitch=67, velocity=92), Note(start=0.848958, end=0.916667, pitch=65, velocity=72), Note(start=0.916667, end=1.036458, pitch=67, velocity=72), Note(start=1.005208, end=1.286458, pitch=65, velocity=84), Note(start=1.296875, end=1.505208, pitch=64, velocity=76), Note(start=1.505208, end=2.005208, pitch=65, velocity=92), Note(start=1.984375, end=2.244792, pitch=73, velocity=96), Note(start=2.244792, end=2.437500, pitch=72, velocity=80), Note(start=2.442708, end=2.687500, pitch=71, velocity=80), Note(start=2.645833, end=2.786458, pitch=70, velocity=76), Note(start=2.666667, end=2.901042, pitch=49, velocity=72), Note(start=2.895833, end=3.375000, pitch=48, velocity=72), Note(start=3.364583, end=3.562500, pitch=55, velocity=84), Note(start=3.572917, end=3.635417, pitch=53, velocity=72), Note(start=3.640625, end=3.760417, pitch=55, velocity=7

In [28]:
len(inputs)

1244

In [41]:
with gen_summary_writer.as_default():
    result = mt.generate(inputs[160:], beam=beam, length=length, tf_board=False)

generating... 0.0% completed
generating... 8.038585209003216% completed
generating... 16.077170418006432% completed
generating... 24.115755627009648% completed
generating... 32.154340836012864% completed
generating... 40.19292604501607% completed
generating... 48.231511254019296% completed
generating... 56.27009646302251% completed
generating... 64.30868167202573% completed
generating... 72.34726688102894% completed
tf.Tensor([193 257 185 ... 256 195 256], shape=(2048,), dtype=int32)


In [42]:
len(result)

2048

In [40]:
decode_midi(inputs[160:], file_path='./result/fourth_inputs.mid')

[<[SNote] time: 0 type: note_off, value: 65, velocity: 0>, <[SNote] time: 0.02 type: time_shift, value: 1, velocity: 0>, <[SNote] time: 0.02 type: note_off, value: 57, velocity: 0>, <[SNote] time: 0.02 type: note_on, value: 56, velocity: 80>, <[SNote] time: 0.03 type: time_shift, value: 0, velocity: 80>, <[SNote] time: 0.03 type: note_on, value: 60, velocity: 80>, <[SNote] time: 0.17 type: time_shift, value: 13, velocity: 80>, <[SNote] time: 0.17 type: note_off, value: 56, velocity: 80>, <[SNote] time: 0.24000000000000002 type: time_shift, value: 6, velocity: 80>, <[SNote] time: 0.24000000000000002 type: note_on, value: 65, velocity: 92>, <[SNote] time: 0.24000000000000002 type: note_on, value: 56, velocity: 88>, <[SNote] time: 0.28 type: time_shift, value: 3, velocity: 88>, <[SNote] time: 0.28 type: note_off, value: 60, velocity: 88>, <[SNote] time: 0.68 type: time_shift, value: 39, velocity: 88>, <[SNote] time: 0.68 type: note_off, value: 65, velocity: 88>, <[SNote] time: 0.68 type: 

In [44]:
decode_midi(result, file_path='./result/fourth_generated_beam4.mid')

[<[SNote] time: 0 type: note_off, value: 65, velocity: 0>, <[SNote] time: 0.02 type: time_shift, value: 1, velocity: 0>, <[SNote] time: 0.02 type: note_off, value: 57, velocity: 0>, <[SNote] time: 0.02 type: note_on, value: 56, velocity: 80>, <[SNote] time: 0.03 type: time_shift, value: 0, velocity: 80>, <[SNote] time: 0.03 type: note_on, value: 60, velocity: 80>, <[SNote] time: 0.17 type: time_shift, value: 13, velocity: 80>, <[SNote] time: 0.17 type: note_off, value: 56, velocity: 80>, <[SNote] time: 0.24000000000000002 type: time_shift, value: 6, velocity: 80>, <[SNote] time: 0.24000000000000002 type: note_on, value: 65, velocity: 92>, <[SNote] time: 0.24000000000000002 type: note_on, value: 56, velocity: 88>, <[SNote] time: 0.28 type: time_shift, value: 3, velocity: 88>, <[SNote] time: 0.28 type: note_off, value: 60, velocity: 88>, <[SNote] time: 0.68 type: time_shift, value: 39, velocity: 88>, <[SNote] time: 0.68 type: note_off, value: 65, velocity: 88>, <[SNote] time: 0.68 type: 